# Bank Stocks — 1‑Day VaR Backtest & Kupiec Test

This notebook takes the return.csv generated by the finance_analysis.ipynb
- 1‑day **parametric** and **historical** Value‑at‑Risk (VaR) at 95% and 99%
- **Exception counting** (days when realized loss exceeds VaR)
- **Kupiec (unconditional coverage) backtest** with p‑values
- Rolling **exceptions plot**



In [6]:
import os, pandas as pd

if 'returns' not in globals():
    # Try to load from disk if you saved it from the EDA notebook
    for path in ('returns.csv', 'data/returns.csv'):
        if os.path.exists(path):
            returns = pd.read_csv(path, index_col=0, parse_dates=True)
            # make sure sorted by date
            returns = returns.sort_index()
            print(f'Loaded returns from {path}:', returns.shape)
            break
    else:
        raise RuntimeError(
            "Couldn't find `returns`. In your EDA notebook, add:\n"
            "    returns.to_csv('returns.csv')\n"
            "Then rerun this notebook."
        )


In [8]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from scipy.stats import norm
from math import log
os.makedirs("figs", exist_ok=True)

assert 'returns' in globals(), "Please define a DataFrame `returns` with daily returns by ticker."

def parametric_var(ret_series, p=0.95, window=250):
    """Rolling parametric VaR using normal assumption."""
    mu = ret_series.rolling(window).mean()
    sigma = ret_series.rolling(window).std(ddof=0)
    z = norm.ppf(1-p)  # e.g., p=0.95 -> 5% left-tail
    var = mu + sigma * z
    return var

def historical_var(ret_series, p=0.95, window=250):
    """Rolling historical VaR: left-tail quantile of last `window` returns."""
    return ret_series.rolling(window).quantile(1-p)

def kupiec_test(exceedances, T, p):
    """Kupiec unconditional coverage LR test. Return (LR_uc, p_value)."""
    # exceedances: integer N (exceptions); T: total trials; p: expected exception rate (e.g., 0.05)
    from scipy.stats import chi2
    N = int(exceedances)
    pi_hat = N / T if T > 0 else 0.0
    if N == 0 or N == T:
        # handle edge cases robustly
        pi_hat = max(min(pi_hat, 1-1e-12), 1e-12)
    L1 = ( (1-p)**(T-N) ) * ( p**N )
    L2 = ( (1-pi_hat)**(T-N) ) * ( (pi_hat)**N )
    LR_uc = -2.0 * np.log(L1/L2)
    p_value = 1 - chi2.cdf(LR_uc, df=1)
    return float(LR_uc), float(p_value)

def backtest_var(ret_series, name, alpha_list=(0.95,0.99), window=250, plot=True):
    results = []
    for p in alpha_list:
        var_p = parametric_var(ret_series, p=p, window=window)
        var_h = historical_var(ret_series, p=p, window=window)
        # Align
        aligned = pd.DataFrame({'r': ret_series, 'VaR_param': var_p, 'VaR_hist': var_h}).dropna()
        # Exceptions: realized return < VaR (loss greater than VaR)
        exc_param = (aligned['r'] < aligned['VaR_param']).sum()
        exc_hist  = (aligned['r'] < aligned['VaR_hist']).sum()
        T = len(aligned)
        LRp, pv_p = kupiec_test(exc_param, T, 1-p)
        LRh, pv_h = kupiec_test(exc_hist,  T, 1-p)
        results.append({
            'ticker': name, 'p': p, 'T': T,
            'exceptions_param': int(exc_param), 'exceptions_hist': int(exc_hist),
            'kupiec_LR_param': LRp, 'kupiec_p_param': pv_p,
            'kupiec_LR_hist':  LRh, 'kupiec_p_hist':  pv_h
        })
        if plot:
            fig, ax = plt.subplots(figsize=(10,3))
            ax.plot(aligned.index, aligned['r'], lw=0.6, label='Return')
            ax.plot(aligned.index, aligned['VaR_param'], lw=0.8, label=f'Param VaR {int(p*100)}%')
            ax.plot(aligned.index, aligned['VaR_hist'],  lw=0.8, label=f'Hist VaR {int(p*100)}%')
            exc_idx = aligned.index[ aligned['r'] < aligned['VaR_hist'] ]
            ax.scatter(exc_idx, aligned.loc[exc_idx,'r'], s=8, label='Exceptions (Hist VaR)')
            ax.set_title(f'{name}: 1-day VaR Backtest @ {int(p*100)}%  (window={window})')
            ax.legend(loc='lower left'); ax.axhline(0, color='k', lw=0.5)
            plt.tight_layout()
            out = f"figs/var_backtest_{name}.png"
            plt.savefig(out, dpi=180)
            plt.close(fig)
    return pd.DataFrame(results)

all_results = []
for col in returns.columns:
    res = backtest_var(returns[col].dropna(), name=col, alpha_list=(0.95,0.99), window=250, plot=True)
    all_results.append(res)

results_df = pd.concat(all_results, ignore_index=True)
results_df


,ticker,p,T,exceptions_param,exceptions_hist,kupiec_LR_param,kupiec_p_param,kupiec_LR_hist,kupiec_p_hist
0,BAC Return,0.95,2267,131,156,2.760885,9.659477e-02,15.195451,0.000097
1,BAC Return,0.99,2267,51,41,26.399005,2.776835e-07,12.077542,0.000510
2,C Return,0.95,2267,145,154,8.580899,3.397076e-03,13.865585,0.000196
3,C Return,0.99,2267,66,40,55.238804,1.066924e-13,10.901122,0.000961
4,GS Return,0.95,2267,133,129,3.404923,6.500214e-02,2.181664,0.139663
5,GS Return,0.99,2267,55,42,33.299952,7.898461e-09,13.303647,0.000265
6,JPM Return,0.95,2267,129,145,2.181664,1.396634e-01,8.580899,0.003397
7,JPM Return,0.99,2267,45,35,17.268716,3.244855e-05,5.809255,0.015942
8,MS Return,0.95,2267,131,136,2.760885,9.659477e-02,4.490517,0.034083
9,MS Return,0.99,2267,53,38,29.772182,4.859146e-08,8.702272,0.003178


In [3]:
# Save results to CSV for your repo
results_df.to_csv('var_results.csv', index=False)
results_df.head()

,ticker,p,T,exceptions_param,exceptions_hist,kupiec_LR_param,kupiec_p_param,kupiec_LR_hist,kupiec_p_hist
0,BAC Return,0.95,2267,131,156,2.760885,9.659477e-02,15.195451,0.000097
1,BAC Return,0.99,2267,51,41,26.399005,2.776835e-07,12.077542,0.000510
2,C Return,0.95,2267,145,154,8.580899,3.397076e-03,13.865585,0.000196
3,C Return,0.99,2267,66,40,55.238804,1.066924e-13,10.901122,0.000961
4,GS Return,0.95,2267,133,129,3.404923,6.500214e-02,2.181664,0.139663


In [4]:
import pandas as pd
res = pd.read_csv('var_results.csv')
# Pick historical VaR (usually better in volatile regimes)
r = (res
     .query("p in [0.95, 0.99]")
     .assign(level=lambda d: (d['p']*100).astype(int).astype(str)+'%')
     .loc[:, ['ticker','level','T','exceptions_hist','kupiec_p_hist']]
     .sort_values(['ticker','level']))
print(r)


        ticker level     T  exceptions_hist  kupiec_p_hist
0   BAC Return   95%  2267              156       0.000097
1   BAC Return   99%  2267               41       0.000510
2     C Return   95%  2267              154       0.000196
3     C Return   99%  2267               40       0.000961
4    GS Return   95%  2267              129       0.139663
5    GS Return   99%  2267               42       0.000265
6   JPM Return   95%  2267              145       0.003397
7   JPM Return   99%  2267               35       0.015942
8    MS Return   95%  2267              136       0.034083
9    MS Return   99%  2267               38       0.003178
10  WFC Return   95%  2267              145       0.003397
11  WFC Return   99%  2267               38       0.003178


## Interpretation Notes

- **Kupiec p-value** tests whether the observed exception rate matches the expected rate (5% for VaR95, 1% for VaR99).
  - p-value **> 0.05** → we *cannot* reject correct coverage (good/green zone).
  - p-value **< 0.05** → exception rate inconsistent with the VaR level (fail/red zone).
- **Historical vs Parametric VaR**: Historical adapts to volatility clusters; Parametric is simpler but can understate risk in fat-tail regimes.

